Importing Libraries

In [ ]:
import cv2
import time 

import tensorflow as tf
import os
import shutil
import numpy as np
import random
from tqdm import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
from pathlib import Path
import re
import cv2
from keras import backend as K
tf.__version__
import glob

#check if tensorflow gpu is being used
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("tensorflow version:", tf.__version__)

In [ ]:
#Realtime Segmentation with Dice Score and Bounding Box Detection

#Reference for bbox(Github): https://github.com/murtazahassan/OpenCV-Python-Tutorials-and-Projects/blob/master/Intermediate/RealTime_Shape_Detection_Contours.py 
#Reference for bbox (Video): https://www.youtube.com/watch?v=Fchzk1lDt7Q 

#Reference for realtime segmentation: https://anirudhtopiwala.com/my-robots/skin-segmentation-using-deep-learning/
#Github Code for realtime segmentation: https://github.com/anirudhtopiwala/abd-skin-segmentation/blob/master/Real%20Time%20Skin%20Segmentation/UNET-live.py 
#playing video on openCV: https://www.geeksforgeeks.org/python-play-a-video-using-opencv/ 
#adding FPS for openCV: https://www.geeksforgeeks.org/python-displaying-real-time-fps-at-which-webcam-video-file-is-processed-using-opencv/ 

#videos to choose from
NeedleViz_path1 = 'Data/edited data/102622_Water.mp4'
NeedleViz_path2 = 'Data/edited data/102822_Water.mp4'

# vc = cv2.VideoCapture(0) #opens camera
vc = cv2.VideoCapture(video_path)

prev_frame_time = 0
new_frame_time = 0
average_fps_list = []

frameWidth = 440
frameHeight = 440
vc.set(3, frameWidth)
vc.set(4, frameHeight)



def empty(a):
  pass

cv2.namedWindow("Parameters")
cv2.resizeWindow("Parameters",640,240)
cv2.createTrackbar("Threshold1","Parameters",155,255,empty)
cv2.createTrackbar("Threshold2","Parameters",191,255,empty)
cv2.createTrackbar("Area","Parameters",5000,30000,empty)


#Reference: https://stackoverflow.com/questions/39308030/how-do-i-increase-the-contrast-of-an-image-in-python-opencv
cv2.createTrackbar("alpha","Parameters", 253, 255, empty)
cv2.createTrackbar("beta","Parameters", 1, 255, empty)


if (vc.isOpened()== False): 
  print("Error opening video  file")

while(vc.isOpened()):
    rval, frame = vc.read()
    
    if rval == True:
      #Insert Code Here

      new_frame_time = time.time()
      fps = 1/(new_frame_time-prev_frame_time)
      prev_frame_time = new_frame_time

      fps = int(fps)
      average_fps_list.append(fps)

      #Processing the frame generation and predicting anatomical features (segmentation)
      ima1 = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
      img = resize(ima1, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
      newframe= np.expand_dims(img, axis=0)
      preds_dark = loadedModelDark.predict(newframe, verbose=0) 
      preds_bright = loadedModelBright.predict(newframe, verbose=0)
      preds_combined = cv2.add(preds_dark, preds_bright) 
      new_preds = np.squeeze(preds_combined, axis=0)   
      new_preds = (new_preds > 0.4).astype(np.uint8) #used to play around with threshold value
      

      preds_resized = resize(new_preds, (440, 440), mode='constant', preserve_range=True) #resizing predicted segmentation for display
        

      realtime_dice = myDiceScore(new_preds, groundTruth_combined, 1) #evaluates dice score of frame vs ground truth image
      realtime_dice = tf.get_static_value(realtime_dice) #gets static value from tensor 
      realtime_dice = round(realtime_dice, 2)#limiting float to two decimal places


      #Converting float64 to uint8 for proper processing in opencv
      updated_maskV2 = preds_resized.astype(np.uint8)
      imgContour = frame.copy() #copy desired frame (normal frame or predicted mask) to have contour and bboxes layed over for realtime display
      

      #smoothens the image 
      imgBlur = cv2.GaussianBlur(updated_maskV2,(7,7), 1)

      #Adjusting contrast levels (issue was that the original image was so dim it could not be seen as mentioned during a warning)
      alpha = cv2.getTrackbarPos("alpha", "Parameters")
      beta = cv2.getTrackbarPos("beta", "Parameters")
      contrast_imageBlur = alpha*imgBlur+beta

      #detects canny edges
      threshold1 = cv2.getTrackbarPos("Threshold1", "Parameters")
      threshold2 = cv2.getTrackbarPos("Threshold2", "Parameters")
      imgCanny = cv2.Canny(contrast_imageBlur, threshold1, threshold2)


      #dilation to further remove any noise generated by canny detection 
      kernel = np.ones((5, 5))
      imgDil = cv2.dilate(imgCanny, kernel, iterations=1)
      getContours(imgDil,imgContour)
      
      
      # putting the FPS count and dice score evalution on the preview frame
      cv2.putText(imgCanny, 'FPS: {}'.format(str(fps)), (7, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
      cv2.putText(imgCanny, 'DiceScore: {}'.format(str(realtime_dice)), (180, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
      

      # SEPERATE WINDOWS #
      # cv2.imshow("preview1", contrast_imageBlur) #originally preds_resized
      # cv2.imshow("preview2", imgCanny) #originally preds_resized
      # cv2.imshow("preview3", imgContour)
      # cv2.imshow("normal",frame)
      
      # COMBINED WINDOWS #
      imgStack = stackImages(0.8,([frame,contrast_imageBlur],[imgCanny,imgContour]))
      cv2.imshow("stacked", imgStack)


      # Press Q on keyboard to  exit
      if cv2.waitKey(1) & 0xFF == ord('q'): #original waitkey is 25
          break
    
    #Break out of loop if video is done
    else:
        break  

vc.release() #Release the video capture object

# Close windows

# cv2.destroyWindow("normal")
# cv2.destroyWindow("preview1")
# cv2.destroyWindow("preview2")
# cv2.destroyWindow("preview3")

cv2.destroyWindow("stacked")

cv2.destroyWindow("Parameters")
